In [ ]:
import os
from typing import Optional
from openai import OpenAI
from dotenv import load_dotenv
from abc import ABC, abstractmethod
import json
import re
from exec_code import execute_code, extract_code_from_string
from prompt import CODE_DEVELOPER_SYSTEMPROMPT, DOCUMENTATION_DEVELOPER_SYSTEMPROMPT, EXECUTE_UIDESIGN_SYSTEMPROMPT
load_dotenv()

client = OpenAI()

class Agent(ABC):
    
    def __init__(self, 
        model: Optional[str] = None, 
        system_prompt: Optional[str] = None
    ) -> None:
        
        model = model if model else os.getenv("OPENAI_DEFAULT_MODEL")
        
        self.memory = []
        self.model = model
        
        if system_prompt:
            self.memory.append({"role": "system", "content": system_prompt})
    
    def add_to_memory(self, role, message):
        self.memory.append({"role": role, "content": message})

    def get_schema(self):
        return self.function
        
    @abstractmethod
    def run(self,prompt):
        """User must define this method. Run the agent"""
    
class ConversationAgent(Agent):
    
    def __init__(self, 
        model: Optional[str] = None, 
        system_prompt: Optional[str] = None,
        name_agent: Optional[str] = None,
        description: Optional[str] = None
    ) -> None:
        super().__init__(model,system_prompt)

        self.name_agent = name_agent
        self.description = description

        self.function={
            "name": self.name_agent,
            "description": self.description,
            "parameters": {
                "type": "object",
                "properties": {
                    "user_prompt": {
                        "type": "string",
                        "description": "User prompt to the agent"
                    }
                },
                "required": ["user_query"]
            }
        }
        
    def run(self,prompt):

        self.add_to_memory("user", prompt)
        completion=''
        stream = client.chat.completions.create(
            messages=self.memory,
            model=self.model,
            stream=True
        )
        print(f"{self.name_agent}:")
        for chunk in stream:
            text_chunk=chunk.choices[0].delta.content
            if text_chunk:
                completion+=text_chunk
                print(text_chunk, end='', flush=True)
        
        self.add_to_memory("assistant", completion)
        
        return completion

In [ ]:

class AgentManager(Agent):
    
    def __init__(self, 
        model: Optional[str] = None, 
        system_prompt: Optional[str] = None,
        list_agents: Optional[list] = None
    ) -> None:
        super().__init__(model,system_prompt)
        
        self.list_agents=list_agents
    
    
    def get_agent_list_schemas(self):
        return [agent.get_schema() for agent in self.list_agents]   
    
    def run(self,prompt):
        
        print(f"----------------- RUN MANAGER -----------------")
        
        #print(self.memory)
        # User query process
        self.add_to_memory("user", prompt)
        print(f"User: {prompt}")
        
        # Choose what agent i have to execute to resolve the problem (Select speaker)
        messages = self.memory + [{"role": "user", "content": f"[PROMPT]\n{prompt}\n\n[AGENTS]\n{self.get_agent_list_schemas()}"}]
        chat_completion = client.chat.completions.create(
            messages=messages,
            model=self.model,
            functions=self.get_agent_list_schemas()
        )
        #print(chat_completion)
                
        # Get the agent to execute
        agent_name=chat_completion.choices[0].message.function_call.name
        agent_prompt=json.loads(chat_completion.choices[0].message.function_call.arguments)['user_prompt']
                
        print(f"Manager: {agent_prompt}")
        self.add_to_memory("user", agent_prompt)
         
        #get the class agent to execute
        class_agent = next((agent for agent in self.list_agents if agent.name_agent == agent_name), None)
        
        #Execute the agent
        for i in range(2):
            response_agent=class_agent.run(agent_prompt)
            
            code_to_execute = extract_code_from_string(response_agent)
            if code_to_execute:
                try:
                    print("Executing code...")
                    exec_response = execute_code(code_to_execute,use_docker=False)
                    if exec_response[0] == 0:
                        print(f"Code executed successfully")
                        code_exec_result=f'{prompt}\n\nCode executed successfully'
                    else:
                        print(f"Error executing code: {exec_response[1]}")
                        code_exec_result=f'{prompt}\n\nError executing code: {exec_response[1]}'
                except Exception as e:
                    print(f"Error executing code: {e}")
                    code_exec_result=f'{prompt}\n\nError executing code: {e}'
            else:
                print(f"No code found to execute.")
                prompt+=f'{prompt}\n\nError executing code: bad format'
        
        #print(f"{agent_name}: {response_agent}")
        
        self.add_to_memory("user", response_agent)
        
        return response_agent

In [ ]:
from prompt import COORDINATOR_SYSTEMPROMPT


CodeDeveloper=ConversationAgent(
    model="gpt-4-1106-preview",
    name_agent="CodeDeveloper",
    description="Agent to create code in python",
    system_prompt=CODE_DEVELOPER_SYSTEMPROMPT
)

UIDesignDeveloper=ConversationAgent(
    model="gpt-4-1106-preview",
    name_agent="UIDesignDeveloper",
    description="Agent to resolve UI design problems in python",
    system_prompt=EXECUTE_UIDESIGN_SYSTEMPROMPT
)

DocumentationDeveloper=ConversationAgent(
    model="gpt-4-1106-preview",
    name_agent="DocumentationDeveloper",
    description="Agent to document code",
    system_prompt=DOCUMENTATION_DEVELOPER_SYSTEMPROMPT
)

# Manager Agents
manager=AgentManager(
    model="gpt-4-1106-preview",
    list_agents=[
        CodeDeveloper,
        DocumentationDeveloper
    ],
    system_prompt=COORDINATOR_SYSTEMPROMPT
)

In [ ]:

manager.run("Create a code for a face detector using my camera, show me it in display and it must run by 30 sec and end.")

In [ ]:
manager.run("I need to develop an additional functionality for the existing code, remove the stop based on time condition. It necessitates the integration of a 'Stop' button which, when activated, will cease the camera operation and visual output. Additionally, the graphic interface must have the features in one specific window.")

In [ ]:

manager.run("All code before, document the code and explain it")